In [1]:
import pandas as pd
import time
print("Loading the data file now, could take a while...")
start_time = time.time()
df = pd.read_csv('../data/Hotel_Reviews.csv')
end = time.time()
print("Loading took" + str(round(end - start_time, 2)) + " seconds")

Loading the data file now, could take a while...
Loading took5.67 seconds


In [2]:
print ("The shap of the data (rows, cols) is " + str(df.shape))

The shap of the data (rows, cols) is (515738, 17)


### Calculate the frequency count for reviewer nationalities:

How many distinct values are there for the column Reviewer_Nationality and what are they?
What reviewer nationality is the most common in the dataset (print country and number of reviews)?

In [ ]:
nationality_freq = df['Reviewer_Nationality'].value_counts()
print("There are " + str(nationality_freq.size) + " different nationalities in the dataset")
print(nationality_freq)

There are 227different nationalities in the dataset
Reviewer_Nationality
United Kingdom               245246
United States of America      35437
Australia                     21686
Ireland                       14827
United Arab Emirates          10235
                              ...  
Cape Verde                        1
Northern Mariana Islands          1
Tuvalu                            1
Guinea                            1
Palau                             1
Name: count, Length: 227, dtype: int64


### What are the next top 10 most frequently found nationalities, and their frequency count?

In [6]:
print("The next 10 highest frequency reviewer nationalities are:")
print(nationality_freq[1:11].to_string())

The next 10 highest frequency reviewer nationalities are:
Reviewer_Nationality
United States of America     35437
Australia                    21686
Ireland                      14827
United Arab Emirates         10235
Saudi Arabia                  8951
Netherlands                   8772
Switzerland                   8678
Germany                       7941
Canada                        7894
France                        7296


### What was the most frequently reviewed hotel for each of the top 10 most reviewer nationalities?

In [7]:
for nat in nationality_freq[:10].index:
   # First, extract all the rows that match the criteria into a new dataframe
   nat_df = df[df["Reviewer_Nationality"] == nat]   
   # Now get the hotel freq
   freq = nat_df["Hotel_Name"].value_counts()
   print("The most reviewed hotel for " + str(nat).strip() + " was " + str(freq.index[0]) + " with " + str(freq[0]) + " reviews.") 

C:\Users\user\AppData\Local\Temp\ipykernel_23868\2970136166.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("The most reviewed hotel for " + str(nat).strip() + " was " + str(freq.index[0]) + " with " + str(freq[0]) + " reviews.")
C:\Users\user\AppData\Local\Temp\ipykernel_23868\2970136166.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("The most reviewed hotel for " + str(nat).strip() + " was " + str(freq.index[0]) + " with " + str(freq[0]) + " reviews.")
C:\Users\user\AppData\Local\Temp\ipykernel_23868\2970136166.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In

The most reviewed hotel for United Kingdom was Britannia International Hotel Canary Wharf with 3833 reviews.
The most reviewed hotel for United States of America was Hotel Esther a with 423 reviews.
The most reviewed hotel for Australia was Park Plaza Westminster Bridge London with 167 reviews.
The most reviewed hotel for Ireland was Copthorne Tara Hotel London Kensington with 239 reviews.


C:\Users\user\AppData\Local\Temp\ipykernel_23868\2970136166.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("The most reviewed hotel for " + str(nat).strip() + " was " + str(freq.index[0]) + " with " + str(freq[0]) + " reviews.")
C:\Users\user\AppData\Local\Temp\ipykernel_23868\2970136166.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("The most reviewed hotel for " + str(nat).strip() + " was " + str(freq.index[0]) + " with " + str(freq[0]) + " reviews.")
C:\Users\user\AppData\Local\Temp\ipykernel_23868\2970136166.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In

The most reviewed hotel for United Arab Emirates was Millennium Hotel London Knightsbridge with 129 reviews.
The most reviewed hotel for Saudi Arabia was The Cumberland A Guoman Hotel with 142 reviews.


C:\Users\user\AppData\Local\Temp\ipykernel_23868\2970136166.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("The most reviewed hotel for " + str(nat).strip() + " was " + str(freq.index[0]) + " with " + str(freq[0]) + " reviews.")
C:\Users\user\AppData\Local\Temp\ipykernel_23868\2970136166.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("The most reviewed hotel for " + str(nat).strip() + " was " + str(freq.index[0]) + " with " + str(freq[0]) + " reviews.")
C:\Users\user\AppData\Local\Temp\ipykernel_23868\2970136166.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In

The most reviewed hotel for Netherlands was Jaz Amsterdam with 97 reviews.
The most reviewed hotel for Switzerland was Hotel Da Vinci with 97 reviews.
The most reviewed hotel for Germany was Hotel Da Vinci with 86 reviews.
The most reviewed hotel for Canada was St James Court A Taj Hotel London with 61 reviews.


C:\Users\user\AppData\Local\Temp\ipykernel_23868\2970136166.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("The most reviewed hotel for " + str(nat).strip() + " was " + str(freq.index[0]) + " with " + str(freq[0]) + " reviews.")
